## Face extraction using visual words

---

In this notebook I will use bag-of-visual generated by the [last notebook](https://colab.research.google.com/drive/1J5B1rTAGaAfFelf8P9d4lXzjjH1j_WBr?usp=sharing) and apply multiple models in order to select the best for bounding box. 

---

references:
  - sklearn for multiregression : [models](https://scikit-learn.org/stable/modules/multiclass.html)
  - save and load models : [post](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)

In [1]:
!gdown 1ptO1oDhO4FLj0GF41yA0FSnM3rx-Mpae

Downloading...
From: https://drive.google.com/uc?id=1ptO1oDhO4FLj0GF41yA0FSnM3rx-Mpae
To: /content/final_sift_data.pkl
100% 286M/286M [00:02<00:00, 133MB/s]


In [2]:
!mkdir models

In [22]:
from sklearn.metrics import classification_report,plot_confusion_matrix,confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from keras.utils.vis_utils import plot_model

from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC 
from sklearn import metrics
import plotly.express as px
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import ast
from numpy import random


tqdm.pandas()
sns.set_theme()
MODEL_DIR_NAME = 'models/'

In [4]:
def validation_metrics(y_pred, y_true, show_img=True):
  rmse = np.sqrt(np.square(y_pred - y_true).mean(axis=-1)) * 200
  fig = px.histogram(rmse)
  fig.show()
  return np.mean(rmse)

In [35]:
def sort_arr(cur_arr, val_ig=-1):
  temp_arr = cur_arr[cur_arr != val_ig].copy()
  temp_arr = np.sort(temp_arr)
  cur_arr[:len(temp_arr)] = temp_arr
  return cur_arr

def generate_random(arr_size=255):
  return random.randint(200, size=(255))

In [36]:
df = pd.read_pickle('final_sift_data.pkl')
# df['visual_words_histogram'] = [np.array(ast.literal_eval(cur_vword)) for cur_vword in df['visual_words_histogram']]
df[['x0', 'y0', 'x1', 'y1']] = df[['x0', 'y0', 'x1', 'y1']] / 200 
df['sift_p_arr_hist'] = df['sift_p_arr_hist'].apply(lambda x: x/sum(x))
df['sift_p_arr_f_p'] = df['sift_p_arr_f_p'].apply(lambda x: sort_arr(x))
df['sift_p_arr_f_p_q'] = df['sift_p_arr_f_p_q'].apply(lambda x: sort_arr(x,0))
df['random'] = [generate_random() for _ in range(len(df))]

df.head(3)

,img_location,x0,y0,x1,y1,sift_p_arr_f_p,sift_p_arr_f_p_q,sift_p_arr_hist,random
0,celeb_data_resized/000001.jpg,0.232274,0.103348,0.784841,0.558952,"[1137, 1691, 1730, 1730, 1880, 2062, 2071, 230...","[14, 22, 22, 22, 24, 27, 27, 30, 32, 35, 37, 4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[43, 70, 142, 65, 178, 98, 25, 6, 198, 93, 145..."
1,celeb_data_resized/000002.jpg,0.170213,0.158249,0.692671,0.673401,"[552, 964, 964, 1143, 1356, 2149, 2149, 2169, ...","[7, 12, 12, 14, 17, 27, 27, 28, 28, 30, 30, 34...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0032467532467...","[87, 142, 137, 32, 117, 60, 140, 149, 183, 29,..."
2,celeb_data_resized/000003.jpg,0.432000,0.209964,0.614000,0.658363,"[2451, 3497, 3787, 4119, 4119, 4285, 4881, 492...","[34, 49, 53, 57, 57, 60, 68, 68, 77, 93, 96, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[126, 97, 63, 29, 67, 176, 54, 112, 84, 8, 11,..."


In [37]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

In [38]:
len(df_train), len(df_test)

(7028, 3012)

In [39]:
x_train = np.array(list(df_train['random'].values))
y_train = np.array(list(df_train[['x0', 'y0', 'x1', 'y1']].values))

x_test = np.array(list(df_test['random'].values))
y_test = np.array(list(df_test[['x0', 'y0', 'x1', 'y1']].values))

In [40]:
def save_keras_model(model_name, model):
  model.save('models/' + model_name)

## Defining the models for bounding box generation

---

Now we define a model predict the bounding boxes of the image based on bag of visual words. 

So first we can try some Keras NN :

 - Simple dense nn
 - Simple dense nn with skip connections


In order to check the model performance we generate the histogram of RMSE of each example, them we display the mean of the RMSE distribution. The perfect model will have a mean RMSE equal to zero and a histogram concentrated around zero.

In [41]:
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from keras.utils.vis_utils import plot_model


#### Simple dense NN

---

In [42]:
def simple_dense_model(dropout=0.5,n_neurons=64, num_classes=4, lr=1e-4):

  inputs = Input(shape=x_train[0].shape[0])
  x = Dense(n_neurons, activation="relu")(inputs)
  x = Dense(n_neurons, activation="relu")(x)
  x = Dropout(dropout)(x)
  outputs = Dense(num_classes, activation="sigmoid")(x)

  ret_model = Model(inputs=inputs, outputs=outputs)

  opt = Adam(learning_rate=lr)
  ret_model.compile(opt, "mean_squared_error")
  return ret_model

In [43]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [44]:
simple_dense = simple_dense_model(dropout=0.7,n_neurons=512, lr=1e-3)
simple_dense.fit(x_train,y_train, batch_size=64, epochs=500, 
                validation_split=0.2, shuffle=True, verbose=True,  callbacks=[callback])

Epoch 1/500
88/88 [==============================] - 1s 4ms/step - loss: 0.1090 - val_loss: 0.0969
Epoch 2/500
88/88 [==============================] - 0s 5ms/step - loss: 0.1053 - val_loss: 0.0969
Epoch 3/500
88/88 [==============================] - 1s 6ms/step - loss: 0.1002 - val_loss: 0.0969
Epoch 4/500
88/88 [==============================] - 1s 6ms/step - loss: 0.1000 - val_loss: 0.0969


In [45]:
save_keras_model('simple_dense', simple_dense)

INFO:tensorflow:Assets written to: models/simple_dense/assets


In [46]:
pred = simple_dense.predict(x_test)
validation_metrics(pred, y_test)

60.04272242671849

### Dense with skip connections

---

In [47]:
def simple_dense_with_skip_model(dropout=0.5,n_neurons=64, num_classes=4, lr=1e-4):

  inputs = Input(shape=x_train[0].shape[0])

  x0 = Dense(n_neurons, activation="relu")(inputs)
  x = Dense(n_neurons, activation="relu")(x0)
  x0 = Add()([x, x0])

  x = Dense(n_neurons, activation="relu")(x0)
  x = Dense(n_neurons, activation="relu")(x)
  x0 = Add()([x, x0])

  x = Dense(n_neurons, activation="relu")(x0)
  x = Dense(n_neurons, activation="relu")(x)
  x0 = Add()([x, x0])

  x = Dropout(dropout)(x0)

  outputs = Dense(num_classes, activation="sigmoid")(x)

  simple_dense_skip = Model(inputs=inputs, outputs=outputs)

  opt = Adam(learning_rate=lr)
  simple_dense_skip.compile(opt, "mean_squared_error")
  return simple_dense_skip

In [48]:
dense_with_skip = simple_dense_with_skip_model(dropout=0.7,n_neurons=512, lr=1e-4)
dense_with_skip.fit(x_train,y_train, batch_size=128, epochs=500, 
                    validation_split=0.1, shuffle=True, verbose=True,  callbacks=[callback])

Epoch 1/500
50/50 [==============================] - 1s 7ms/step - loss: 0.1587 - val_loss: 0.0943
Epoch 2/500
50/50 [==============================] - 0s 4ms/step - loss: 0.1004 - val_loss: 0.0943
Epoch 3/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0989 - val_loss: 0.0943
Epoch 4/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0991 - val_loss: 0.0943


In [49]:
save_keras_model('dense_with_skip', dense_with_skip)

INFO:tensorflow:Assets written to: models/dense_with_skip/assets


In [50]:
pred = dense_with_skip.predict(x_test)
validation_metrics(pred, y_test)

60.04272242671849

In [51]:
!zip -r models_NN_for_face_extraction.zip models

updating: models/ (stored 0%)
updating: models/dense_with_skip/ (stored 0%)
updating: models/dense_with_skip/assets/ (stored 0%)
updating: models/dense_with_skip/keras_metadata.pb (deflated 92%)
updating: models/dense_with_skip/variables/ (stored 0%)
updating: models/dense_with_skip/variables/variables.data-00000-of-00001 (deflated 13%)
updating: models/dense_with_skip/variables/variables.index (deflated 70%)
updating: models/dense_with_skip/saved_model.pb (deflated 90%)
updating: models/simple_dense/ (stored 0%)
updating: models/simple_dense/assets/ (stored 0%)
updating: models/simple_dense/keras_metadata.pb (deflated 87%)
updating: models/simple_dense/variables/ (stored 0%)
updating: models/simple_dense/variables/variables.data-00000-of-00001 (deflated 15%)
updating: models/simple_dense/variables/variables.index (deflated 61%)
updating: models/simple_dense/saved_model.pb (deflated 88%)
